COMP 215 - LAB 3 (NEO)
----------------
#### Name: Ben Blair
#### Date: 23/01/2023

As usual, the first code cell simply imports all the modules we'll be using...

In [28]:
import datetime, json, requests
from dataclasses import dataclass
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

## Fetch Complete Data for One Asteroid (from Lab 2)

Notice that the record for each `neo` is a dictionary with `id` field that uniquely identifies this record in the database.

We can use this `id` to fetch complete orbital and close approach data for the NEO.

For example, this query fetches the complete data set for the asteroid with the given id...


In [29]:
API_KEY = 'GT5a6VBQVVTnT8H1jSPfja2DfDMaC1d0UsYeWsYW'

id = '2226554' # Asteroid ID taken from Lab 2
url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}/?api_key={API_KEY}'
response = requests.request("GET", url, headers={}, data={})
data = json.loads(response.text)

pprint(data)

{'absolute_magnitude_h': 19.64,
 'close_approach_data': [{'close_approach_date': '1900-10-30',
                          'close_approach_date_full': '1900-Oct-30 21:14',
                          'epoch_date_close_approach': -2182819560000,
                          'miss_distance': {'astronomical': '0.4183489529',
                                            'kilometers': '62584112.270570323',
                                            'lunar': '162.7377426781',
                                            'miles': '38887964.1060755374'},
                          'orbiting_body': 'Earth',
                          'relative_velocity': {'kilometers_per_hour': '30027.9637577316',
                                                'kilometers_per_second': '8.3411010438',
                                                'miles_per_hour': '18658.2089249083'}},
                         {'close_approach_date': '1901-06-26',
                          'close_approach_date_full': '1901-Jun-26 20:27

## Lab 3 - Exercise 1
### Define an Asteroid Class

Using what we learned in the textbook, define a simple `Asteroid` class with some basic state variables for a single NEO.  Your Asteroid class should define at least 4 "state variables”:

*   `id`, `name`, `estimated_diameter`, and `is_potentially_hazardous`
*   Provide an `__init__(self, ...)` method to initialize a new Asteroid object with specific data values and a little code to test your class.
*   Add a `__str__(self)` method that returns a nicely formatted string representation of the object, plus a little code to test it.

In [30]:
@dataclass
class Asteroid:
    """ Represents an asteroid. """
    id: int
    name: str
    estimated_diameter: float # Units: meters
    is_potentially_hazardous: bool

    # Replace __init__ and __str__ methods by using @dataclass
    # def __init__(self, id, name, diameter, hazardous):
    #     self.id = id
    #     self.name = name
    #     self.diameter = diameter
    #     self.hazardous = hazardous
    
    # def __str__(self):
    #     return f'Asteroid ID: {self.id}, Name: {self.name}, Est. Diameter (m): {self.diameter:.2f}, Hazardous (T/F): {self.hazardous}'

test_asteroid = Asteroid('1234567', '1234567 2023 (BB31)', 555.55, True)
print(test_asteroid)

Asteroid(id='1234567', name='1234567 2023 (BB31)', estimated_diameter=555.55, is_potentially_hazardous=True)


## Lab 3 - Exercise 2
### Factory method: Asteroid.from_NEO

We want to be able to construct Asteroid objects easily from the NEO API.

Define a `“@classmethod”: from_NEO(cls, neo_id)` that takes the id for a single NEO, fetches the NEO record from the API, then constructs and returns an Asteroid object representing that NEO. This kind of method is sometimes called a “factory” – it constructs an object from raw materials.


In [31]:
@dataclass
class Asteroid:
    """ Represents an asteroid. """
    id: int
    name: str
    estimated_diameter: float # Units: meters
    is_potentially_hazardous: bool

    @classmethod
    def from_NEO(cls, neo_id):
        url = f'https://api.nasa.gov/neo/rest/v1/neo/{neo_id}/?api_key={API_KEY}'
        response = requests.request("GET", url, headers={}, data={})
        data = json.loads(response.text)

        # Pull the desired attributes out of the data
        name = data['name']
        # Get max and min estimated diameter in meters
        max_diameter = float(data['estimated_diameter']['meters']['estimated_diameter_max'])
        min_diameter = float(data['estimated_diameter']['meters']['estimated_diameter_min'])
        # Average the max and min diameter to get estimated diameter in meters
        estimated_diameter = (max_diameter + min_diameter) / 2
        is_potentially_hazardous = data['is_potentially_hazardous_asteroid']

        return cls(neo_id, name, estimated_diameter, is_potentially_hazardous)

test_asteroid_from_id = Asteroid.from_NEO('2226554')
print(test_asteroid_from_id)

Asteroid(id='2226554', name='226554 (2003 WR21)', estimated_diameter=507.62454924335003, is_potentially_hazardous=True)


## Lab 3 - Exercise 3
### Define a CloseApproach class

Each NEO comes with a list of “`close_approach_data`”, where each record in this list represents a single “close approach” to another orbiting body.

*   Develop a class named “**CloseApproach**” to represent a single close approach record. State variables are **asteroid** (Asteroid object),  **orbiting body** (`str`), **approach date** (`datetime` object!),  **miss distance** (`float` choose units, document it, and be consistent!),  and **relative velocity** (ditto).
*   Define a Factory class method to construct a `CloseApproach` object from one close approach data record (a dictionary object).

Remember to parse the date/time string into a `datetime` object.

In [32]:
@dataclass
class CloseApproach:
    """ Represents a single close approach record. """
    asteroid: Asteroid
    orbiting_body: str
    approach_date: datetime
    miss_distance: float     # Units: kilometers
    relative_velocity: float # Units: kilometers per hour

    @classmethod
    # Method must take an Asteroid object and a close approach data record
    def from_record(cls, asteroid, record):
        # Pull the desired attributes from the supplied record
        body = record['orbiting_body']
        # Get date of approach and convert to a datetime object
        date = datetime.datetime.strptime(record['close_approach_date'], '%Y-%m-%d').date()
        # Get miss distance in kilometers
        distance = float(record['miss_distance']['kilometers'])
        # Get relative velocity in kilometers per hour
        velocity = float(record['relative_velocity']['kilometers_per_hour'])

        return cls(asteroid, body, date, distance, velocity)

test_approach = {
    'close_approach_date': '1900-10-30',
    'close_approach_date_full': '1900-Oct-30 21:14',
    'epoch_date_close_approach': -2182819560000,
    'miss_distance': {'astronomical': '0.4183489529',
                    'kilometers': '62584112.270570323',
                    'lunar': '162.7377426781',
                    'miles': '38887964.1060755374'},
    'orbiting_body': 'Earth',
    'relative_velocity': {'kilometers_per_hour': '30027.9637577316',
                        'kilometers_per_second': '8.3411010438',
                        'miles_per_hour': '18658.2089249083'}
}

# Use test asteroid from Exercise 2 and sample data record to test from_record factory class method
CloseApproach.from_record(test_asteroid_from_id, test_approach)

CloseApproach(asteroid=Asteroid(id='2226554', name='226554 (2003 WR21)', estimated_diameter=507.62454924335003, is_potentially_hazardous=True), orbiting_body='Earth', approach_date=datetime.date(1900, 10, 30), miss_distance=62584112.27057032, relative_velocity=30027.9637577316)

## Lab 3 - Exercise 4
### Add list of close approaches to Asteroid

Every Asteroid should have a list of “close approaches”.  Add a new state variable to your Asteroid class, initially an empty list.  In the factory method, use a list comprehension to build a list of `CloseApproach` objects for the Asteroid from the NEO data record.

Extend your test code to demonstrate this new feature.

In [35]:
from operator import attrgetter

# Test list comprehension before adding it into the Asteroid class factory method
close_approaches = [CloseApproach.from_record(test_asteroid_from_id, approach) for approach in data['close_approach_data']]
num_approaches = len(close_approaches)

print(f"Close approaches found: {num_approaches}")

sorted_close_approaches = sorted(close_approaches, key=attrgetter('miss_distance'))
pprint(sorted_close_approaches)

Close approaches found: 130
[CloseApproach(asteroid=Asteroid(id='2226554', name='226554 (2003 WR21)', estimated_diameter=507.62454924335003, is_potentially_hazardous=True), orbiting_body='Earth', approach_date=datetime.date(1972, 6, 25), miss_distance=4303822.052007232, relative_velocity=32284.6643092962),
 CloseApproach(asteroid=Asteroid(id='2226554', name='226554 (2003 WR21)', estimated_diameter=507.62454924335003, is_potentially_hazardous=True), orbiting_body='Earth', approach_date=datetime.date(1901, 6, 26), miss_distance=4619550.017178589, relative_velocity=33429.0363049198),
 CloseApproach(asteroid=Asteroid(id='2226554', name='226554 (2003 WR21)', estimated_diameter=507.62454924335003, is_potentially_hazardous=True), orbiting_body='Earth', approach_date=datetime.date(2102, 6, 28), miss_distance=4775418.551175805, relative_velocity=32339.6089540488),
 CloseApproach(asteroid=Asteroid(id='2226554', name='226554 (2003 WR21)', estimated_diameter=507.62454924335003, is_potentially_haza

In [25]:
@dataclass
class Asteroid:
    """ Represents an asteroid. """
    id: int
    name: str
    estimated_diameter: float # Units: meters
    is_potentially_hazardous: bool
    close_approaches: list # Add attribute to store list of close approaches

    @classmethod
    def from_NEO(cls, neo_id):
        url = f'https://api.nasa.gov/neo/rest/v1/neo/{neo_id}/?api_key={API_KEY}'
        response = requests.request("GET", url, headers={}, data={})
        data = json.loads(response.text)

        # Pull the desired attributes out of the data
        name = data['name']
        # Get max and min estimated diameter in meters
        max_diameter = float(data['estimated_diameter']['meters']['estimated_diameter_max'])
        min_diameter = float(data['estimated_diameter']['meters']['estimated_diameter_min'])
        # Average the max and min diameter to get estimated diameter in meters
        estimated_diameter = (max_diameter + min_diameter) / 2
        is_potentially_hazardous = data['is_potentially_hazardous_asteroid']
        close_approaches = [] # Initially set close_approaches to an empty list

        # Generate Asteroid object
        asteroid = cls(neo_id, name, estimated_diameter, is_potentially_hazardous, close_approaches)
        # Populate close_approaches list with CloseApproach objects using list comprehension and from_record factory method
        asteroid.close_approaches = [CloseApproach.from_record(asteroid, approach) for approach in data['close_approach_data']]

        return asteroid

test_asteroid_from_id = Asteroid.from_NEO('2226554')
print(test_asteroid_from_id)

# Check that the number of close approaches matches the total found in the test comprehension above
assert len(test_asteroid_from_id.close_approaches) == num_approaches
print(len(test_asteroid_from_id.close_approaches), "close approaches associated with Asteroid", str(test_asteroid_from_id.id))

Asteroid(id='2226554', name='226554 (2003 WR21)', estimated_diameter=507.62454924335003, is_potentially_hazardous=True, close_approaches=[CloseApproach(asteroid=..., orbiting_body='Earth', approach_date=datetime.date(1900, 10, 30), miss_distance=62584112.27057032, relative_velocity=30027.9637577316), CloseApproach(asteroid=..., orbiting_body='Earth', approach_date=datetime.date(1901, 6, 26), miss_distance=4619550.017178589, relative_velocity=33429.0363049198), CloseApproach(asteroid=..., orbiting_body='Earth', approach_date=datetime.date(1907, 1, 8), miss_distance=18554355.766636536, relative_velocity=24041.8524771157), CloseApproach(asteroid=..., orbiting_body='Earth', approach_date=datetime.date(1907, 9, 6), miss_distance=62042216.3364171, relative_velocity=26520.1966604789), CloseApproach(asteroid=..., orbiting_body='Venus', approach_date=datetime.date(1909, 8, 6), miss_distance=19705248.981622554, relative_velocity=7812.3404746925), CloseApproach(asteroid=..., orbiting_body='Earth'

## Lab 3 - Optional
### Take your skills to the next level!

With this data structure in place, we can now start answering all kinds of interesting questions about a single Asteroid or a set of Asteroids.

Here’s a couple ideas to try:

*   Add a  method to the `Asteroid` class, `closest_earth_approach`, that returns the `CloseApproach` object that represents the closest approach the Asteroid makes to Earth.

*   Write a function named `most_dangerous_approach`, that takes a date range and returns a single “potentially hazardous” Asteroid object that makes the closest approach to Earth in within that range. Your algorithm will ultimately need to:

   *   grab the list of NEO’s for the given date range
   *   use a list comprehension to build the list of Asteroid objects for the NEOs returned
   *   use a list comprehension to **filter** potentially hazardous Asteroids only
   *   use a list comprehension to **map** each Asteroid to its `closest_earth_approach`
   *   apply Python’s `min` function to identify the Asteroid with the `closest_earth_approach`

   You may want to *decompose* some of these steps into smaller functions.

*   Add a method to the `Asteroid` class, `estimated_mass`, that computes an estimate of the Asteroid’s mass based on its diameter.  This is a model – state your assumptions.

*   Add a method to the `CloseApproach` class, `impact_force`,  that estimates the force of impact if the Asteroid hit the orbiting object.  Again, this is a model, state your assumptions.

In [38]:
# Add a closest_earth_approach method

@dataclass
class Asteroid:
    """ Represents an asteroid. """
    id: int
    name: str
    estimated_diameter: float # Units: meters
    is_potentially_hazardous: bool
    close_approaches: list # Add attribute to store list of close approaches

    @classmethod
    def from_NEO(cls, neo_id):
        url = f'https://api.nasa.gov/neo/rest/v1/neo/{neo_id}/?api_key={API_KEY}'
        response = requests.request("GET", url, headers={}, data={})
        data = json.loads(response.text)

        # Pull the desired attributes out of the data
        name = data['name']
        # Get max and min estimated diameter in meters
        max_diameter = float(data['estimated_diameter']['meters']['estimated_diameter_max'])
        min_diameter = float(data['estimated_diameter']['meters']['estimated_diameter_min'])
        # Average the max and min diameter to get estimated diameter in meters
        estimated_diameter = (max_diameter + min_diameter) / 2
        is_potentially_hazardous = data['is_potentially_hazardous_asteroid']
        close_approaches = [] # Initially set close_approaches to an empty list

        # Generate Asteroid object
        asteroid = cls(neo_id, name, estimated_diameter, is_potentially_hazardous, close_approaches)
        # Populate close_approaches list with CloseApproach objects using list comprehension and from_record factory method
        asteroid.close_approaches = [CloseApproach.from_record(asteroid, approach) for approach in data['close_approach_data']]

        return asteroid
    
    def closest_earth_approach(self):
        # Filter the approaches by orbiting body
        earth_approaches = [approach for approach in self.close_approaches if approach.orbiting_body == 'Earth']
        
        # Find the closest approach to Earth
        closest_to_earth = earth_approaches[0]
        for approach in earth_approaches:
            if approach.miss_distance < closest_to_earth.miss_distance:
                closest_to_earth = approach
        return closest_to_earth

test_asteroid = Asteroid.from_NEO('2226554')
closest_earth_approach = test_asteroid.closest_earth_approach()

# Check that the closest_earth_approach method returns the expected result
expected = sorted_close_approaches[0].miss_distance
result = closest_earth_approach.miss_distance
assert result == expected

print(f"The closest Asteroid {test_asteroid.id} will come to Earth is {closest_earth_approach.miss_distance:.0f} kilometers.")

The closest Asteroid 2226554 will come to Earth is 4303822 kilometers.


In [ ]:
# Write a function most_dangerous_approach



In [ ]:
# Add an estimated_mass method



In [ ]:
# Add an impact_force method

